0.1 라이브러리 임포트

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.autograd import Variable
import numpy as np

0.2 하이퍼파라미터 셋팅

In [2]:
# Hyper-parameters
batch_size= 128
num_epochs = 10
word_vec_size = 256
dropout_p = 0.3
hidden_size = 512
num_layers = 4

## yhk 추가
learning_rate = 0.001 # 디폴트 0.001

In [3]:
# Device configuration 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

1. SMS train, test dataset 가져오기

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
cd /content/drive/My Drive/Colab Notebooks/NLP

/content/drive/My Drive/Colab Notebooks/NLP


In [7]:
!pip install torchtext==0.4.0

     |████████████████████████████████| 61kB 4.8MB/s 
  Found existing installation: torchtext 0.10.0
    Uninstalling torchtext-0.10.0:
      Successfully uninstalled torchtext-0.10.0


In [8]:
from data_loader import DataLoader

In [9]:
loaders = DataLoader(
    train_fn='./sms.maxlen.uniq.shuf.train.tsv',
    batch_size=batch_size,
    valid_ratio=.2,
    device=-1,
    max_vocab=999999,
    min_freq=5,
)

In [11]:
test_loaders = DataLoader(
    train_fn='./sms.maxlen.uniq.shuf.train.tsv',
    batch_size=batch_size,
    valid_ratio=.01,
    device=-1,
    max_vocab=999999,
    min_freq=5,
)

2. 대략적인 데이터 형태

In [12]:
print("|train| = ", len(loaders.train_loader.dataset),
      '|valid| = ', len(loaders.valid_loader.dataset))

vocab_size = len(loaders.text.vocab)
num_classes = len(loaders.label.vocab)
print('|vocab| = ', vocab_size, '|classes| = ', num_classes)

|train| =  3722 |valid| =  931
|vocab| =  1567 |classes| =  2


3. 데이터 로드 함수
학습시킬 때 batch_size 단위로 끊어서 로드하기 위함

In [13]:
n = 3 
for i, data in enumerate(loaders.train_loader):
  labels = data.label
  texts = data.text

  if i > n:
    break
  print("[%d]" %i)
  print("한 번에 로드되는 데이터 크기:", len(labels))

  # 출력
  for j in range(n):
    label = labels[j].numpy()
    text = texts[j].numpy()
    print("label:", label)
    print("text:", text.shape)

[0]
한 번에 로드되는 데이터 크기: 128
label: 0
text: (15,)
label: 0
text: (15,)
label: 0
text: (15,)
[1]
한 번에 로드되는 데이터 크기: 128
label: 0
text: (6,)
label: 0
text: (6,)
label: 0
text: (6,)
[2]
한 번에 로드되는 데이터 크기: 128
label: 0
text: (5,)
label: 0
text: (5,)
label: 0
text: (5,)
[3]
한 번에 로드되는 데이터 크기: 128
label: 0
text: (16,)
label: 0
text: (16,)
label: 0
text: (16,)


4. 모델 선언

In [14]:
# Recurrent neural network (many-to-one)
class RNN(nn.Module):
  def __init__(self,
               input_size, # vocab_size
               word_vec_size, # word embedding vector 차원
               hidden_size, # bidirectional LSTM 의 hidden state & cell state의 size
               n_classes,
               num_layers=4, # 쌓을 레이어 개수
               dropout_p=0.3
               ):
    super(RNN, self).__init__()

    self.input_size = input_size
    self.word_vec_size = word_vec_size
    self.hidden_size = hidden_size
    self.n_classes = num_classes
    self.num_layers = num_layers
    self.dropout_p = dropout_p

    # 입력 차원(vocab_size), 출력 차원(word_vec_size)
    self.emb = nn.Embedding(input_size, word_vec_size)

    self.lstm = nn.LSTM(input_size=word_vec_size,
                        hidden_size=hidden_size,
                        num_layers=num_layers,
                        dropout=dropout_p,
                        batch_first=True,
                        bidirectional=True)
    self.fc = nn.Linear(hidden_size*2, num_classes)
    # LogSoftmax + NLLLOSS instead of Softmax + CrossEntropy
    self.activation = nn.LogSoftmax(dim=-1) # 마지막차원에 softmax 씌워줌

  def forward(self, x):
    # x: (batch_size, length)
    x = self.emb(x)

    # x: (batch_size, length, word_vec_size)
    x, _ = self.lstm(x) # x: output, _: 마지막 timestep의 hidden state & cell state

    # x: (batch_size, length, hidden_size * 2)
    # x[:,-1]: (batch_size,1,hidden_size*2)
    out = self.activation(self.fc(x[:,-1])) # 마지막 time step
    # self.fc(x[:,-1]): (batch_size, num_classes)

    return out

In [15]:
model = RNN(input_size=vocab_size,
            word_vec_size=word_vec_size,
            hidden_size=hidden_size,
            n_classes=num_classes,
            num_layers=num_layers,
            dropout_p=dropout_p)

In [16]:
def ComputeAccr(dloader, imodel):
  correct = 0
  total = 0

  model.eval() # test mode
  for i, data in enumerate(dloader): # batch_size 만큼
    texts = data.text.to(device) # (batch_size, length)
    labels = data.label.to(device) # (batch_size, num_classes)

    # Forward prop.
    output = model(texts) # (batch_size, num_classes)
    _, output_index = torch.max(output,1) # (batch_size, 1)

    total += labels.size(0)
    correct += (output_index == labels).sum().float()
  # print("Accuracy of Test Data: {}".format(100*correct/total))

  model.train()
  return (100*correct/total).numpy()

In [17]:
print("Accuracy of Test Data: %.2f" %ComputeAccr(loaders.valid_loader, model))

Accuracy of Test Data: 13.64


5. loss, optimizer

In [18]:
# Loss and optimizer
# loss_func= nn.CrossEntropyLoss()
loss_func = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

6. 학습

In [ ]:
# Train the model
total_step = len(loaders.train_loader)
for epoch in range(num_epochs):
  for i, data in enumerate(loaders.train_loader):
    texts = data.text.to(device) # (batch_size, length)
    labels = data.label.to(device) # (batch_size, num_classes)

    print("[%d]" %i)

    # Forward prop.
    outputs = model(texts)
    loss = loss_func(outputs, labels)

    # Backward prop. & optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1)%10 == 0:
      print('Epoch [{}/{}], Step[{}/{}], Loss: {:.4f}, Accr: {:.2f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item(),
                                                                            ComputeAccr(loaders.valid_loader, model)))

[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
Epoch [1/10], Step[10/30], Loss: 0.1667, Accr: 86.36
[10]
[11]
[12]
[13]
[14]
[15]
[16]
[17]
[18]
[19]
Epoch [1/10], Step[20/30], Loss: 0.3385, Accr: 86.36
[20]
[21]
[22]
[23]
[24]
[25]
[26]
[27]
[28]
[29]
Epoch [1/10], Step[30/30], Loss: 0.1719, Accr: 86.36
[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
Epoch [2/10], Step[10/30], Loss: 0.1885, Accr: 86.36
[10]
[11]
[12]
[13]
[14]


7.테스트

In [ ]:
print("Accuracy of Valid Data: %.2f" %ComputeAccr(test_loaders.train_loader, model))

Accuracy of Valid Data: 97.31


8. 학습된 파라미터 저장

In [ ]:
netname = '/rnn_weight.pkl'
torch.save(model, netname,)


9. 학습된 파라미터 로드

실무에서 학습된(pretraining) 파라미터 로드하고 싶다면: 5,6,8 과정 생략한 채 실행

In [ ]:
netnME = './rnn_weight.pkl'
model = torch.load(netname)
print("Accuracy of Valid Data: %.2f" %ComputeAccr(loaders.valid_loader, model))

Accuracy of Valid Data: 97.31
